In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
validation_csv = '.../bats-data/bat_counting_error_quantification_with_counts_padded.csv'
dark_csv = '.../bats-data/bat_counting_error_quantification_dark.csv'
# falloff_csv = '/home/golden/Dropbox/bats-data/falloff-validation-test.csv'

In [ ]:
val_df = pd.read_csv(validation_csv)
# val_df = pd.read_csv(falloff_csv)

In [ ]:
dark_df = pd.read_csv(dark_csv)

In [ ]:
dark_df.head()

In [ ]:
print(len(dark_df.columns.values))
print(len(val_df.columns.values))

In [ ]:
dark_inds = dark_df['number-missed-detections-new'].notnull()
print(np.sum(dark_inds))

In [ ]:
val_df.loc[dark_inds, 'total_bats'] = (
    (val_df.loc[dark_inds, 'new_method_count_going'] 
     - pd.to_numeric(dark_df.loc[dark_inds, 'false-detections-moving-up-blue-new'], errors='coerce')
    )
    - (val_df.loc[dark_inds, 'new_method_count_coming'] 
       - pd.to_numeric(dark_df.loc[dark_inds, 'number-false-detections-moving-down-yellow-new'], errors='coerce')
      )
    + pd.to_numeric(dark_df.loc[dark_inds, 'number-missed-detections-new'], errors='coerce')
)

In [ ]:
# val_df.head()

In [ ]:
# val_df['camera-name'] = [s.split('-')[2] for s in val_df['Video clip name'].values]
# val_df['clip-num'] = [s.split('-')[4] for s in val_df['Video clip name'].values]

In [ ]:
total_bat_values = np.sum(~np.isnan(val_df['total_bats']))
old_bat_values = np.sum(~np.isnan(val_df['total_bats_old_method']))
new_bat_values = np.sum(~np.isnan(val_df['total_bats_new_method']))
print(f'total bat values: {total_bat_values} '
     + f'old bat values: {old_bat_values} '
     + f'new bat values: {new_bat_values}')

In [ ]:
print(val_df['total_bats_old_method'].sum())
print(val_df.loc[~np.isnan(val_df['total_bats']), 'total_bats_old_method'].sum())

In [ ]:
val_df.loc[val_df['camera_name']=='MusoleTower']

In [ ]:
# ind = 19
# # val_df.loc[ind, 'Number missed detections'] = 0
# # val_df.loc[ind, 'Number false detections moving up (blue)'] = 0
# # val_df.loc[ind, 'Number false detections moving down (yellow)'] = 0
# # val_df.loc[ind, 'total_bats'] = 0
# val_df.loc[ind]

In [ ]:
observation_groups = val_df.loc[~np.isnan(val_df['total_bats'])].groupby(['date_folder', 'camera-name'])

old_frac = []
new_frac = []

for key,item in observation_groups:
    a_group = observation_groups.get_group(key)
    a_group = a_group.sort_values('clip-num').reset_index()
    fig, [ax1, ax2] = plt.subplots(2, 1)
    ax1.plot(a_group['total_bats'], label='total')
    ax1.plot(a_group['total_bats_old_method'], label='old')
    ax1.plot(a_group['total_bats_new_method'], label='new')
    ax1.legend()
    cumsum_total = np.cumsum(a_group['total_bats'])
    cumsum_old = np.cumsum(a_group['total_bats_old_method'])
    cumsum_new = np.cumsum(a_group['total_bats_new_method'])
    ax2.plot(cumsum_total, label='total')
    ax2.plot(cumsum_old, label='old')
    ax2.plot(cumsum_new, label='new')
    frac_seen_new = cumsum_new.iloc[-1] / cumsum_total.iloc[-1]
    frac_seen_old = cumsum_old.iloc[-1] / cumsum_total.iloc[-1]
    old_frac.append(frac_seen_old)
    new_frac.append(frac_seen_new)
    camera = a_group['camera-name'].iloc[0]
    date = a_group['date_folder'].iloc[0]
    fig.suptitle(f"{date}, {camera}: new {frac_seen_new}, old {frac_seen_old}", fontsize=14)
    ax2.legend()
    
print('old', np.mean(old_frac), 'new', np.mean(new_frac))

In [ ]:
a_group['camera-name'].iloc[0]

In [ ]:
frame_root_folder = '.../Elements/bats'

In [ ]:
import glob
import os

date = '17Nov'
camera_name = 'FibwePublic'

frame_files = sorted(
                glob.glob(os.path.join(frame_root_folder, 
                                       date,
                                       camera_name, 
                                       "*", 
                                       "*.jpg")
                         )
            )